In [ ]:
from collections import deque 

def slidingPuzzle(start_state):
    # Convert input list to string representation
    target = "01234"
    start = ''.join(map(str, start_state))
    
    if start == target:
        print("Already at goal!")
        return 0
        
    # valid moves for each position (1D board with 5 positions)
    adj = {
        0: [1, 3],
        1: [0, 2, 4],
        2: [1],
        3: [0, 4],
        4: [1, 3],
    }
    
    # to track which state you've explored
    visited = set()
    visited.add(start)
    
    # Queue holds (current_state_string, index_of_blank, moves)
    q = deque([(start, start.index('0'), 0)])
    
    while q:
        state, pos0, moves = q.popleft()
        
        if state == target:
            print(f"Number of moves: {moves}")
            return moves
            
        for neighbor in adj[pos0]:
            new_state = list(state)
            new_state[pos0], new_state[neighbor] = new_state[neighbor], new_state[pos0]
            s_str = ''.join(new_state)
            
            if s_str not in visited:
                visited.add(s_str)
                q.append((s_str, neighbor, moves + 1))
                
    print("Unsolvable puzzle.")
    return -1

# Enhanced test cases with expected results
tests = [
    ([0, 1, 2, 3, 4], "Already solved", 0),
    ([1, 0, 2, 3, 4], "One move away", 1),
    ([1, 2, 0, 3, 4], "Two moves away", 2),
    ([2, 1, 0, 3, 4], "Three moves away", 3),
    ([3, 1, 2, 4, 0], "Scrambled but solvable", 5),
    ([1, 2, 3, 0, 4], "Harder scramble", 4),
    ([4, 3, 2, 1, 0], "Worst case (far from goal)", 8),
]

print("\n===== SLIDING PUZZLE SOLUTION TESTER =====")
for board, description, expected in tests:
    print(f"\n🔷 Test: {board} — {description}")
    print(f"Expected moves: {expected}")
    result = slidingPuzzle(board)
    if result == expected:
        print(f"✅ PASS: Got {result} moves as expected")
    else:
        print(f"❌ FAIL: Expected {expected} moves, got {result}")

print("\n===== TESTING COMPLETE =====")

In [ ]:
import nbformat
from nbformat import v4 as nbf
from pathlib import Path

# Create a new notebook object
nb = nbf.new_notebook()
cells = []

# Title and introduction
cells.append(nbf.new_markdown_cell(
    "# Sliding Puzzle (5-Tile) Search Algorithms\n\n"
    "This notebook demonstrates the use of search algorithms to solve a simplified 5-tile sliding puzzle.\n\n"
    "We apply various search strategies such as BFS, DFS, and A* to find a path from the initial state to the goal state."
))

# Puzzle Representation & Environment Setup
cells.append(nbf.new_markdown_cell("## Puzzle Representation"))
cells.append(nbf.new_code_cell(
    """\
# A state is represented as a tuple in row-major order.
# The state below is a 5-tile sliding puzzle:
# [3][ ][2]
# [4][1]
# '0' represents the blank tile.
initial_state = (3, 0, 2,
                 4, 1)

# Define the goal state.
# In this example, the goal state is set to:
# [1][2][3]
# [4][ ]
goal_state = (1, 2, 3,
              4, 0)

def print_state(state):
    # Print state in a 2-row, 3-column grid format.
    for i in range(0, len(state), 3):
        row = state[i:i+3]
        print(' '.join(str(x) if x != 0 else ' ' for x in row))
    print()

# Let's see the initial and goal states:
print("Initial State:")
print_state(initial_state)
print("Goal State:")
print_state(goal_state)
"""
))

# Helper Functions
cells.append(nbf.new_markdown_cell("## Helper Functions"))
cells.append(nbf.new_code_cell(
    """\
def get_blank_index(state):
    return state.index(0)

def swap(state, i, j):
    # Swap tiles at indices i and j and return a new state (as tuple)
    state = list(state)
    state[i], state[j] = state[j], state[i]
    return tuple(state)

def get_successors(state):
    successors = []
    idx = get_blank_index(state)
    # The grid is 2 rows x 3 columns
    row, col = divmod(idx, 3)
    # Moves: up, down, left, right
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for dr, dc in moves:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 2 and 0 <= new_col < 3:
            new_idx = new_row * 3 + new_col
            successors.append(swap(state, idx, new_idx))
    return successors

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path
"""
))

# Breadth-First Search Implementation
cells.append(nbf.new_markdown_cell("## Breadth-First Search (BFS)"))
cells.append(nbf.new_code_cell(
    """\
from collections import deque

def bfs(start, goal):
    frontier = deque([start])
    came_from = {}
    visited = set()
    while frontier:
        current = frontier.popleft()
        if current == goal:
            return reconstruct_path(came_from, current)
        visited.add(current)
        for neighbor in get_successors(current):
            if neighbor not in visited and neighbor not in frontier:
                frontier.append(neighbor)
                came_from[neighbor] = current
    return None

# Run BFS and print the solution path
print("BFS Path:")
path_bfs = bfs(initial_state, goal_state)
if path_bfs:
    for state in path_bfs:
        print_state(state)
else:
    print("No solution found!")
"""
))

# Depth-First Search Implementation
cells.append(nbf.new_markdown_cell("## Depth-First Search (DFS)"))
cells.append(nbf.new_code_cell(
    """\
def dfs(start, goal):
    frontier = [start]
    came_from = {}
    visited = set()
    while frontier:
        current = frontier.pop()
        if current == goal:
            return reconstruct_path(came_from, current)
        if current not in visited:
            visited.add(current)
            for neighbor in get_successors(current):
                if neighbor not in visited:
                    frontier.append(neighbor)
                    came_from[neighbor] = current
    return None

# Run DFS and print the solution path
print("DFS Path:")
path_dfs = dfs(initial_state, goal_state)
if path_dfs:
    for state in path_dfs:
        print_state(state)
else:
    print("No solution found!")
"""
))

# A* Search Implementation with Manhattan Distance Heuristic
cells.append(nbf.new_markdown_cell("## A* Search (Using Manhattan Distance Heuristic)"))
cells.append(nbf.new_code_cell(
    """\
def manhattan_distance(state, goal):
    total = 0
    # Calculate Manhattan distance for each numbered tile (ignoring the blank)
    for num in range(1, 5):
        idx1, idx2 = state.index(num), goal.index(num)
        x1, y1 = divmod(idx1, 3)
        x2, y2 = divmod(idx2, 3)
        total += abs(x1 - x2) + abs(y1 - y2)
    return total

def astar(start, goal):
    frontier = [(manhattan_distance(start, goal), 0, start)]
    came_from = {}
    cost_so_far = {start: 0}
    while frontier:
        _, cost, current = heapq.heappop(frontier)
        if current == goal:
            return reconstruct_path(came_from, current)
        for neighbor in get_successors(current):
            new_cost = cost + 1  # Each move has a cost of 1
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + manhattan_distance(neighbor, goal)
                heapq.heappush(frontier, (priority, new_cost, neighbor))
                came_from[neighbor] = current
    return None

# Run A* and print the solution path
print("A* Path:")
path_astar = astar(initial_state, goal_state)
if path_astar:
    for state in path_astar:
        print_state(state)
else:
    print("No solution found!")
"""
))

# Assign all cells to the notebook
nb['cells'] = cells

# Save the notebook to a file
output_path = Path("Sliding_Puzzle_Search.ipynb")
with open(output_path, "w") as f:
    nbformat.write(nb, f)

print(f"Notebook saved to {output_path}")
